In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import nltk
from gensim.models import Word2Vec
import numpy as np

In [ ]:

train_df = pd.read_csv('../res/data_all_train.csv', header=0)
test_df = pd.read_csv('../res/data_all_test.csv', header=0)
validation_df = pd.read_csv('../res/data_all_validate.csv', header=0)

In [ ]:
documents = []
labels=[]
for label in train_df.subreddit:
    labels.append(label)
for doc in train_df.text:
    unicode_string = nltk.word_tokenize(doc.decode('utf8'))
    documents.append(unicode_string)
    
print(len(documents))

print documents[0]

In [ ]:
from collections import defaultdict
from tqdm import tqdm
def load_model():
    glove_input_filename = '../res/glove.twitter.27B.100d.txt'
    glove_output_filename = '../pickle_files/word2vec_glove.pkl'
    # glove_filename = '../res/glove.840B.300d.txt'
    import os.path
    if not os.path.isfile(glove_output_filename):
        word2vec_dict = {}
        word2vec_dict = defaultdict(lambda: [0] * 100, word2vec_dict)

        print("preparing to load word2vec file")
        with open(glove_input_filename) as f:
            for line in tqdm(f, total=2196017):
                love = line.split()
                word, nums = love[0], [float(x) for x in love[1:]]
                word2vec_dict[word] = nums

    return word2vec_dict
model =load_model()

In [ ]:
twod_doc_array=[]
for doc in documents:
    word_array=[]
    i=0
    while(i<len(doc) ):
        if doc[i] in model:
            word_array.append(list(model[doc[i]]))
        i+=1
    i=0
    while (i<40):
        word_array.append(np.zeros(100))
        i+=1
    
    twod_doc_array.append(word_array[:40])

In [ ]:
twod_doc_array=np.array( twod_doc_array)

print twod_doc_array.shape
x=np.reshape(twod_doc_array,(twod_doc_array.shape[0],twod_doc_array.shape[1],twod_doc_array.shape[2],1))
print x.shape

In [ ]:
def cnn_model_fn(features, labels, mode):
    
    # Input Layer

    input_layer = tf.reshape(features['x'], [-1, 40, 100, 1])
    input_layer =tf.cast(input_layer,dtype=tf.float32)

    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    conv3 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
    
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    
    conv4 = tf.layers.conv2d(
      inputs=pool3,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    
    conv5 = tf.layers.conv2d(
      inputs=input_layer,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2)
    conv6 = tf.layers.conv2d(
      inputs=pool5,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    
    pool6 = tf.layers.max_pooling2d(inputs=conv6, pool_size=[2, 2], strides=2)
    # Dense Layer
    
    pool2_flat = tf.reshape(pool2, [-1, 10 * 25 * 64])
    pool4_flat= tf.reshape(pool4, [-1, 10 * 25 * 64])
    pool6_flat= tf.reshape(pool6, [-1, 10 * 25 * 64])
    
    poolflat=tf.concat([pool2_flat,pool4_flat,pool6_flat],0)
    poolflat=tf.reshape(poolflat, [-1, 3 * 10 * 25 * 64])
    print poolflat.get_shape().as_list()
    dense = tf.layers.dense(inputs=poolflat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
    inputs=dense, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(1e-3)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
labellist=list(set(labels))

In [ ]:
print labellist

In [ ]:
y=[]
for i in labels:
    y.append(labellist.index(i))
print len(y)
y=np.array(y)
x=np.array(x)

In [ ]:
import time

In [ ]:
start = time.time()

In [ ]:
classifier = tf.estimator.Estimator(
model_fn=cnn_model_fn, model_dir="/tmp/pretrainedmode2")


# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={'x':x},
  y=y,
  batch_size=100,num_epochs=None,
  shuffle=True)

classifier.train(input_fn=train_input_fn,steps=5000)



In [ ]:
end=time.time()
print end-start
start=time.time()

In [ ]:

# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": x},
  y=y,
  num_epochs=1,
  shuffle=False)
eval_results = classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

In [ ]:
end=time.time()
print end-start

In [ ]:
testdocuments = []
testlabels=[]
for label in test_df.subreddit:
    testlabels.append(label)
for doc in test_df.text:
    unicode_string = nltk.word_tokenize(doc.decode('utf8'))
    testdocuments.append(unicode_string)

In [ ]:
test_twod_doc_array=[]
for doc in testdocuments:
    word_array=[]
    i=0
    while(i<len(doc) ):
        if doc[i] in model:
            word_array.append(list(model[doc[i]]))
        i+=1
    i=0
    while (i<40):
        word_array.append(np.zeros(100))
        i+=1
    
    test_twod_doc_array.append(word_array[:40])

test_twod_doc_array=np.array( test_twod_doc_array)
print len(testdocuments)
print test_twod_doc_array.shape
testx=np.reshape(test_twod_doc_array,(test_twod_doc_array.shape[0],test_twod_doc_array.shape[1],test_twod_doc_array.shape[2],1))
print testx.shape

In [ ]:
test_y=[]
for i in testlabels:
    test_y.append(labellist.index(i))
print len(test_y)
test_y=np.array(test_y)
testx=np.array(testx)

In [ ]:
print test_y.shape

In [ ]:
print testx.shape

In [ ]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": testx},
  y=test_y,
  num_epochs=1,
  shuffle=False)
eval_results = classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)